# ノード数変更のスケジュールを設定する

---

workerノード数を増減させるスケジュール設定を行います。

## 概要

workerノードの起動数を変更するスケジュールを設定します。利用頻度の低い時間帯にworkerノードの起動数を減らすことで、クラウドプロバイダの課金や使用電力量を抑えるようにします。

このnotebookで設定を行う対象を次図に示します。

![構成図](images/cw-954-01.png)

workerノード数を制御するコンテナをmanagerノードで実行し、VCノードの起動数の変更やDocker Swarmの状態変更を行います。制御コンテナはsystemd timerから一定時間（１５分）毎に起動されます。起動されたコンテナでは現在のworkerノードの状態を確認し、変更が必要な場合のみVCノードの起動、停止やDocker SwarmのノードAvailabilityを更新します。

### workerノード数

workerノードの利用頻度は授業時間とそれ以外、また日中と夜間、平日と週末など時間帯によって異なることが考えられます。このnotebookでは授業などを行う特定の時間帯とそれ以外の時間帯のノード数を変更する設定を行います。

特定の時間帯でのノード数の指定は「[3.1 スケジュールの指定](#スケジュールの指定)」で行います。時間帯の指定は日毎、週毎、月毎、年毎、特定日時のいずれかで行うことができます。またスケジュールの指定は複数行うことができます。スケジュールで指定した以外の時間帯のノード数の指定を「[3.2 通常時のノード数](#通常時のノード数)」で行います。

このnotebookでスケジュール設定を行なった後に休講など臨時のスケジュールを追加する場合は「812-ノード数のスケジュール設定を変更する.ipynb」を実行してスケジュールの変更を行なってください。

### ノード停止の準備期間

Docker SwarmのノードAvailabilityを[Pause](https://docs.docker.com/engine/swarm/manage-nodes/#change-node-availability)にすることで、そのノードで新たなコンテナが開始されないようにすることができます。このnotebookで構築した環境ではノードを停止する前の一定時間、ノードAvailabilityをPauseにするように設定できます。停止前にPauseにする時間を「[3.3 新たなタスクの受付を停止する期間の指定](#新たなタスクの受付を停止する期間の指定)」で指定してください。

### VCノードの停止方法

workerノードはVCノードとして管理されています。そのためVCPSDKを利用することでノードの起動、停止などの操作を実現できます。VCノードの停止方法には、ノードを削除する方法とノードの電源オフにする方法があります。どちらの方法を用いてノードの停止を行うのかを「[2.3 ノードの停止方法](#ノードの停止方法)」で指定することができます。

> VCノードの電源操作は一部のクラウドプロバイダのみ利用可能な機能です。電源操作をサポートしているかについては[OCSユーザーズマニュアル](https://nii-gakunin-cloud.github.io/ocs-docs/usermanual/appendix.html#vcp%E3%81%AB%E3%81%8A%E3%81%91%E3%82%8B%E3%82%AF%E3%83%A9%E3%82%A6%E3%83%89%E3%83%97%E3%83%AD%E3%83%90%E3%82%A4%E3%83%80%E3%81%A8%E3%82%AA%E3%83%97%E3%82%B7%E3%83%A7%E3%83%8A%E3%83%AB%E6%A9%9F%E8%83%BD%E3%81%AE%E5%AF%BE%E5%BF%9C)を確認してください。

### 想定するシナリオ

このNotebookで想定するCoursewareHubの利用状況を示します。

* 授業時間中
  * 授業開始数十分前から授業終了数十分後まで
  * workerノードを最大限利用する時間帯
* 授業時間後
  * 授業終了数十分後から数時間後まで
  * 利用者が徐々に減少していく時間帯
* 授業時間外
  * 授業時間中、授業時間後以外の時間
  * 利用者が少ない時間帯

#### 授業時間中

![授業時間中](images/cw-951-02.png)

「授業時間中」は全てのworkerノードを利用可能な状態にします。この時間帯はworkerノードの計算資源の上限まで、生徒が利用するJupyter環境(single-user Jupyter notebook server)コンテナを起動することができるようにします。

#### 授業時間後

![授業時間中後](images/cw-951-03.png)

授業が終了した後の一定時間（授業時間外となるまで）は一部のノードでDocker SwarmのAvailabilityを[Pause](https://docs.docker.com/engine/swarm/manage-nodes/#change-node-availability)に変更します。この時間帯を「授業時間後」と呼びます。Pauseに変更したノードでは実行中のコンテナを引き続き利用できますが、新たなコンテナが起動できなくなります。また「[5. CoursewareHubのculling設定](#CoursewareHubのculling設定)」を設定している場合、アイドル状態のNotebookサーバーは順次停止されます。これらの設定により実行するコンテナは少数の利用可能状態のノードに集約されることが期待されます。

#### 授業時間外

![授業時間外](images/cw-951-04.png)

「授業時間外」になると必要最小限のノード以外を停止します。ノードの停止方法は電源オフにする方法とノードを削除する方法を選択できます。どちらの方法をとる場合もクラウドプロバイダの課金を抑えられることが期待されます。

停止するノードで実行しているコンテナは「授業時間後」の時間帯に全て終了することが期待されます。もし停止するノードで実行中のコンテナがある場合、ノード停止にともない実行中のコンテナはいったん終了します。その後、利用可能なworkerノードで再実行されます（計算資源に空きがある場合）。

#### パラメータの指定方法 



「授業時間中」と「授業時間後」を合わせた時間帯に利用するノード数を「[3.1 スケジュールの指定](#スケジュールの指定)」で指定します。また「授業時間後」にする時間を「[3.3 新たなタスクの受付を停止する期間の指定](#新たなタスクの受付を停止する期間の指定)」で指定します。「授業時間外」のノード数は「[3.2 通常時のノード数](#通常時のノード数)」で指定します。



例えば「授業時間中」の時間帯が月曜日の10:00から12:00で「授業時間後」の時間帯がその後の12:00から16:00の場合を考えます。授業中に起動するノード数を5とし、それ以外の時間帯のノード数を1とする場合以下のような指定を行います。

```python
schedule_list = [
    {"begin": "Mon 10:00", "end": "Mon 16:00", "node_count": 5}
]                               # 3.1 スケジュールの指定

schedule_default_nodes = 1      # 3.2 通常時のノード数

schedule_drain_time = 60 * 4    # 3.3 新たなタスクの受付を停止する期間の指定
```


## パラメータの指定

VCノードのスケールアウト、スケールインを行うのに必要となるパラメータを指定します。

### UnitGroup名

CoursewareHubのUnitGroup名を指定します。

VCノードを作成時に指定したUnitGroup名を確認するために `group_vars` ファイル名の一覧を表示します。

In [ ]:
!ls -1 --hide all group_vars/

UnitGroup名を指定してください。

In [ ]:
# (例)
# ugroup_name = 'CoursewareHub'

ugroup_name = 

#### チェック

指定されたグループ名が妥当なものであることをチェックします。

managerノードにアクセスできることを確認します。

In [ ]:
target_group = f'{ugroup_name}_manager'
!ansible {target_group} -m ping

managerノードを管理者として操作できることを確認します。

In [ ]:
!ansible {target_group} -b -a 'whoami'

指定されたUnitGroupで、既にスケジュール設定が行われていないことを確認します。

In [ ]:
%run scripts/group.py
gvars = load_group_vars(ugroup_name)
if gvars.get("vcnode_schedule") is not None:
    raise RuntimeError("既にスケジュール設定が行われています")

> 設定済みのスケジュールを変更するには「812-ノード数のスケジュール設定を変更する.ipynb」を実行してください。

### VCCアクセストークンの入力

VCノードを操作するためにVC Controller(VCC)のアクセストークンが必要となります。
次のセルを実行すると表示される入力枠にVCCのアクセストークンを入力してください。

> アクセストークン入力後に Enter キーを押すことで入力が完了します。

In [ ]:
from getpass import getpass
vcc_access_token = getpass()

入力されたアクセストークンが正しいことを、実際にVCCにアクセスして確認します。

In [ ]:
from common import logsetting
from vcpsdk.vcpsdk import VcpSDK

vcp = VcpSDK(vcc_access_token)

上のセルの実行結果がエラーとなり以下のようなメッセージが表示されている場合は、入力されたアクセストークンに誤りがあります。

```
ERROR - config vc failed: http_status(403)
ERROR - 2025/04/01 00:00:00 UTC: VCPAuthException: xxx:token lookup is failed: permission denied
```

エラーになった場合はこの節のセルを全て `unfreeze` してから、もう一度アクセストークンの入力を行ってください。

### ノードの停止方法

ノードの停止方法は、ノードを削除する方法と、ノードの電源をオフにする方法の２つの方法から選択できます。

次のセルで`deleted`(削除)と`power_down`(電源オフ)のいずれかを指定してください。

In [ ]:
# (例)
# schedule_down_type = "deleted"           # ノードを削除する
# schedule_down_type = "power_down"  # ノードの電源をオフする

schedule_down_type = 

### IPアドレス

workerノードに割り当てるIPアドレスを指定します。workerノードとして起動する最大ノード数のIPアドレスを指定する必要があります。

現在のworkerノードの状態を確認します。

In [ ]:
ug = vcp.get_ugroup(ugroup_name)
ut = ug.get_unit("worker")
ut.df_nodes()

IPアドレスのリストを次のセルで指定してください。現在起動しているworkerノードのIPアドレスが含まれていない場合は、現在のworkerノードを削除した後に改めて指定されたIPアドレスのworkerノードが起動されます。

In [ ]:
# (例)
# vcnode_all_ipaddress = [
#    "172.30.2.101",
#    "172.30.2.102",
#    "172.30.2.103",
#]
#
# vcnode_all_ipaddress = [
#    f"172.30.2.{x}"
#    for x in range(101, 105)
# ]

vcnode_all_ipaddress = [
    
]

### 保存

ここまで指定されたパラメータをgroup_varsに保存します。ただしVCCアクセストークンは保存対象から除外しています。

保存する前に、簡単なチェックを行います。

In [ ]:
%run scripts/check_params.py
%run scripts/group.py
gvars = load_group_vars(ugroup_name)
current_nodes = ut.find_ip_addresses(node_state="RUNNING")

check_parameters(
    "schedule_down_type",
    "vcnode_all_ipaddress",
    params={
        "vcp": vcp,
        "vc_provider": gvars.get("vc_provider"),
        "current_nodes": current_nodes,
    },
    nb_vars=locals(),
)

group_varsに保存します。

In [ ]:
update_group_vars(
    ugroup_name,
    schedule_down_type=schedule_down_type,
    vcnode_all_ipaddress=vcnode_all_ipaddress,
)

## スケジュール定義

### スケジュールの指定

workerノード数を変更する期間とそのノード数を指定します。

ひとつのスケジュールを定義するには、期間を`begin`(開始時刻)と`end`(終了時刻)で指定し、その期間におけるworkerノード数を`node_count`で指定します。この３つの値をpythonのdictで指定します。例えば毎日22:00から翌朝7:00まではworkerノード数を1にするスケジュールは以下のような値で定義します。

```python
{
    "begin": "22:00",
    "end": "7:00",
    "node_count": 1,
}
```

期間は日毎、週毎、月毎、年毎、特定日時の指定が行えます。

日毎の場合は`22:00`のように`{時}:{分}`の形式で指定します。

週毎の場合は`Sat 0:00`のように`{曜日} {時}:{分}`の形式で指定します。曜日は`Sunday`、`日曜日`、`Sun`、`日`のような値で指定を行います。

月毎の場合は`1日 0:00`のように`{日} {時}:{分}`の形式で指定します。日は`1日`、`1`のような値で指定を行います。

年毎の場合は`12/29 0:00`のように`{月}/{日} {時}:{分}`の形式で指定します。

特定日時の場合は`2025/4/1 0:00`のように`{年}/{月}/{日} {時}:{分}`の形式で指定します。

期間の開始と終了は同じ粒度（日毎、週毎、月毎、年毎、特定日時）を指定する必要があります。

次のセルでスケジュール定義のリストを指定してください。リストの個々の要素は上で説明した`begin`, `end`, `node_count`をキーに持つdictになります。複数のスケジュール期間が重複した場合、リストで先に指定したものが優先されます。

In [ ]:
schedule_list = [
# (例)
#    {"begin": "9:00", "end": "16:00", "node_count": 5},                                        # 日毎のスケジュール 9:00--16:00
#    {"begin": "Mon 9:00", "end": "Fri 20:00", "node_count": 5},                          # 週毎のスケジュール 月曜日9:00--金曜日20:00
#    {"begin": "1日 0:00", "end": "1日 12:00", "node_count": 5},                         # 月毎のスケジュール 1日0:00--1日12:00
#    {"begin": "12/29 0:00", "end": "1/4 0:00", "node_count": 0},                     # 年毎のスケジュール 12月29日0:00--1月4日0:00
#    {"begin": "2025/4/1 0:00", "end": "2025/4/2 0:00", "node_count": 5},   # 特定日時のスケジュール 2025/4月1日0:00--2025/4月2日0:00

]

設定されたスケジュールをチェックするための準備を行います。

In [ ]:
from pathlib import Path

!ansible-playbook -v -e venv_dir={str(Path.cwd() / ".venv")} playbooks/setup-local-venv.yml

リスト`schedule_list`に指定したスケジュールをチェックします。次のセルを実行してエラーにならないことを確認してください。指定されたスケジュールのリストに問題ない場合は設定した内容を表示します。意図したスケジュールが指定できていることを確認してください。

In [ ]:
import sys
%run scripts/schedule.py

params = {name: value for name, value in vars().items() if name.startswith("schedule")}
try:
    description_schedule(ugroup_name, params, "node_count")
except ValueError as e:
    print(f"ERROR: {e}", file=sys.stderr)
    raise

### 通常時のノード数

通常時のworkerノード数を指定します。現在のworkerノード数を通常時のノード数とする場合はこの節をスキップして次の節に進むことができます。

現在のworkerノード数を確認します。

In [ ]:
print(gvars["worker_nodes"])

通常時のworkerノード数を次のセルで指定してください。

In [ ]:
# (例)
# schedule_default_nodes = 1

schedule_default_nodes = 

### 新たなタスクの受付を停止する期間の指定

ノードを停止する前に、タスクの受付を停止し新たなコンテナを起動できないようにする期間を設けます。これは、Docker Swarmに対してノードのAvailabilityを`Pause`にすることで実現します。

Availabilityを`Pause`にする時間を次のセルで指定してください。指定の単位は分となります。

In [ ]:
# (例)
# schedule_drain_time = 60

schedule_drain_time = 

### 設定内容の確認

ここまで設定した内容を確認します。

In [ ]:
%run scripts/schedule.py
params = {name: value for name, value in vars().items() if name.startswith("schedule")}
description_schedule(ugroup_name, params)

### 保存

定義したスケジュールをgroup_varsに保存します。

In [ ]:
%run scripts/schedule.py
params = {name: value for name, value in vars().items() if name.startswith("schedule")}
update_group_vars(
    ugroup_name,
    vcnode_schedule=get_schedule_definition(ugroup_name, params),
)

## 配備

スケジュール設定に関する資材をmanagerノードに配備してサービスを開始するように設定します。

次のセルを実行すると資材の配備とサービスの開始を行います。

In [ ]:
import json
from pathlib import Path
from tempfile import TemporaryDirectory

with TemporaryDirectory() as work_dir:
    vars_path = Path(work_dir) / "params.json"
    with vars_path.open(mode="w") as f:
        json.dump({"vcc_access_token": vcc_access_token}, f)
    !ansible-playbook -Dv -l {target_group} -e @{str(vars_path)} playbooks/setup-schedule.yml

CoursewareHubのmanagerノードに配備した資材を確認します。

In [ ]:
!ansible {target_group} -b -a 'tree chdir=/srv/worker-schedule'

In [ ]:
!ansible {target_group} -b -m shell -a 'ls -l cwh* chdir=/etc/systemd/system/'

開始したsystemdタイマーの状態を確認します。通常は１５分ごとにworkerノード状態の確認とノード数の更新を行います。

In [ ]:
!ansible {target_group} -b -a 'systemctl list-timers cwh-worker-node.timer'

## CoursewareHubのculling設定

CoursewareHubを構成するJupyterHubコンテナに環境変数を設定することでculling設定を有効にすることができます。「CoursewareHubのセットアップ.ipynb」でCoursewareHub環境を構築した時にculling機能を有効にしていない場合は、必要に応じてこの章を実行してください。

### パラメータの指定

アイドル状態のsingle-userサーバーを自動的に停止する機能に関するパラメータを指定します。

ここで指定できるパラメータを以下に示します。

|項目 | 説明 | デフォルト値|
|:---|:---|--:|
|cull_server              | アイドル状態のNotebookサーバーの停止を有効／無効化 (yes/no/0/1)| no |
|cull_server_idle_timeout | アイドル状態のNotebookサーバーの停止までのタイムアウト時間(秒)     | 600 |
|cull_server_max_age      | アイドル状態でなくてもNotebookサーバーを停止するまでの時間(秒)     | 0 |
|cull_server_every       | Notebookのアイドル状態のタイムアウトのチェック間隔(秒)             | 0 |

各パラメータの値を以下のセルで指定してください。デフォルト値のままで良いパラメータはセルを実行せずにスキップしてください。

In [ ]:
# (例)
# cull_server = "yes"       # 自動停止を有効にする
# cull_server = "no"        # 自動停止を無効にする

cull_server = "yes"

In [ ]:
# (例)
# cull_server_idle_timeout = 10 * 60               # １０分間
# cull_server_idle_timeout = 12 * 60 * 60      # 12時間

cull_server_idle_timeout = 

In [ ]:
# (例)
# cull_server_max_age = 0
# cull_server_max_age = 24 * 60 * 60      # 24時間

cull_server_max_age = 

In [ ]:
# (例)
# cull_server_every = 0
# cull_server_every = 60

cull_server_every = 

#### パラメータの保存

ここまで指定したパラメータを ansible の変数として `group_vars`ファイルに保存します。

In [ ]:
import yaml
from pathlib import Path

gvars_path = Path(f'group_vars/{ugroup_name}')
with gvars_path.open() as f:
    gvars = yaml.safe_load(f)

jupyterhub_param_names = [
    'cull_server',
    'cull_server_idle_timeout',
    'cull_server_max_age',
    'cull_server_every',
]

for name in jupyterhub_param_names:
    if name in vars():
        gvars[name] = vars()[name]
    elif 'name' in gvars:
        del(gvars['name'])

with gvars_path.open(mode='w') as f:
    yaml.safe_dump(gvars, stream=f)
    
!cat group_vars/{ugroup_name}

### docker-compose.ymlの配置

構築環境にculling設定を追加したdocker-compose.ymlを配置します。

まずはチェックモードで確認を行います。

In [ ]:
!ansible-playbook -l {target_group} -CDv playbooks/deploy-docker-compose.yml || true

実際に`docker-compose.yml`の配置を行います。

In [ ]:
!ansible-playbook -l {target_group} playbooks/deploy-docker-compose.yml

### コンテナの更新

指定されたパラメータに更新された`docker-compose.yml`でコンテナを起動し直します。

コンテナ設定の更新を行います。culling設定を変更している場合はjupyterhubコンテナが起動され直します。

In [ ]:
!ansible {target_group} -a 'chdir={{{{compose_dir}}}} \
    docker stack deploy -c docker-compose.yml coursewarehub'

コンテナの起動状態を確認します。

In [ ]:
!ansible {target_group} -a 'docker service ls'